<a href="https://colab.research.google.com/github/JacquesBLR/Causal_Inference/blob/main/Impact_d%E2%80%99un_Programme_de_Fid%C3%A9lisation_sur_la_R%C3%A9tention_Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contexte et problématique marketing
Une enseigne de e-commerce lance un **programme de fidélité** (carte de fidélité, adhésion premium,
etc.) pour accroître la rétention de ses clients. Avant le déploiement complet, **un test pilote de type A/B test**
a été mené : un échantillon de clients a été **randomisé** en deux groupes – **groupe test** (ayant accès au
programme de fidélité) et **groupe témoin** (sans accès) – afin de mesurer l’effet causal du programme
sur le comportement client. Le critère principal est le **taux de rétention** des clients (pourcentage de
clients toujours actifs/apportant des achats après une période donnée, par ex. 6 mois).
Le défi est d’estimer **l’impact causal** du programme de fidélisation sur la rétention. Le test A/B, en tant
que **expérience randomisée contrôlée (RCT)**, est le **gold standard** pour estimer un effet causal car la
randomisation équilibre les facteurs de confusion entre groupes.  
Nous analyserons les résultats de
cet A/B test de façon fréquentiste (test statistique classique) puis avec une **approche bayésienne** pour
l’interprétation probabiliste. Nous discuterons également des alternatives si la randomisation n’était pas
possible (par ex. usage de matching pour comparer adhérents volontaires vs non-adhérents).


# Données

- anciennete : Ancienneté du client (en mois) avant l’étude.
- groupe : Indique si le client était dans le groupe “Programme” (traitement) ou “Témoin” (contrôle).
- retenu : Indique si le client est resté actif après 6 mois (1 = retenu, 0 = churn).

In [3]:
import numpy as np
import pandas as pd
np.random.seed(42) # fixer la graine pour reproductibilité
# Paramètres simulés
N = 1000
# Ancienneté aléatoire entre 1 et 36 mois
anciennete = np.random.randint(1, 37, size=N)
# Assignation aléatoire au groupe Programme (1) ou Témoin (0)
groupe = np.random.binomial(1, 0.5, size=N)
# Probabilités de rétention hypothétiques (meilleures pour le groupe fidélité)
base_retenu = 0.5 + 0.01 * anciennete # prob. de rétention sans programme
effet_programme = 0.10
# le programme ajoute ~10 points de rétention
prob_retenu = base_retenu + effet_programme * groupe
prob_retenu = np.clip(prob_retenu, 0, 0.999) # borne entre 0 et 99.9%
# Simulation du statut de rétention
retenu = np.random.binomial(1, prob_retenu)
# Construire le DataFrame
df = pd.DataFrame({
'anciennete': anciennete,
'groupe': np.where(groupe==1, 'Programme', 'Témoin'),
'retenu': retenu
})
# Calculer le taux de rétention par groupe
taux_retenu = df.groupby('groupe')['retenu'].mean()
print("Taux de rétention Groupe Témoin : {:.1%}".format(taux_retenu['Témoin']))
print("Taux de rétention Groupe Programme : {:.1%}".format(taux_retenu['Programme']))
print("Différence absolue : {:.1f} points".format((taux_retenu['Programme']-
taux_retenu['Témoin'])*100))

Taux de rétention Groupe Témoin : 67.8%
Taux de rétention Groupe Programme : 78.4%
Différence absolue : 10.6 points


Ici, ~67.8% des clients témoins sont retenus après 6 mois, contre ~78.4% des clients avec la carte de
fidélité, soit un **gain d’environ +10.6 points de rétention grâce au programme.**

## Analyse statistique (fréquentiste) :  
On teste $H_0$ : “le programme n’a aucun effet sur la
rétention” (différence = 0) contre $H_1$ : “le programme améliore la rétention”.  
On peut appliquer un
test de proportion ou un chi² d’indépendance entre groupe et retenu.  
Compte tenu des effectifs, on peut utiliser une approximation normale (test Z).

In [4]:
import statsmodels.api as sm
# Effectifs et conversions par groupe
conv_programme = df[df['groupe']=='Programme']['retenu'].sum()
n_programme = (df['groupe']=='Programme').sum()
conv_témoin = df[df['groupe']=='Témoin']['retenu'].sum()
n_témoin = (df['groupe']=='Témoin').sum()
# Test de différence de proportions (approx. z)
count = np.array([conv_programme, conv_témoin])
nobs = np.array([n_programme, n_témoin])
stat, p_value = sm.stats.proportions_ztest(count, nobs, alternative='larger')
print("Statistique Z = {:.2f}, p-value = {:.4f}".format(stat, p_value))

Statistique Z = 3.77, p-value = 0.0001


Cela indique une différence hautement significative (p < 0.001). **On rejette $H_0$** et on conclut que le
programme de fidélité augmente significativement la rétention.  
Environ +10 points de pourcentage de rétention, c’est un effet substantiel (à confirmer sur la durée et la rentabilité).

**Vérification par régression** : Même si la randomisation garantit en principe l’équilibre des caractéristiques,
on peut peaufiner l’estimation via une **régression logistique de *retenu* sur *groupe* en contrôlant
l’*anciennete*** .  
Cela permet d’estimer l’odd ratio de rétention et de tester l’effet en tenant compte
d’une éventuelle légère différence résiduelle de profil.

In [2]:
import numpy as np
import statsmodels.formula.api as smf

# Régression logistique: retenu ~ groupe + anciennete
model = smf.logit(formula='retenu ~ groupe + anciennete',
data=df.replace({'groupe':{'Témoin':0,'Programme':1}}))
result = model.fit(disp=0)
print(result.params)

Intercept   -0.268928
groupe       0.617461
tenure       0.055803
dtype: float64


/tmp/ipython-input-2-3790170026.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data=df.replace({'groupe':{'Témoin':0,'Programme':1}}))


Ici, **le coefficient de *groupe* (0.62) positif et significatif confirme que l’appartenance au programme
augmente la probabilité de rétention** (odd ratio ~ 1.86, soit les chances de rétention multipliées par ~ 1.86) même en contrôlant l’ancienneté.  
L’effet estimé en probabilité (~+11 points pour un client moyen)
concorde avec le calcul brut. Ceci rassure que le résultat n’est pas dû à un déséquilibre initial des
groupes.

## Approche bayésienne : interprétation plus intuitive pour le décideur
Quelle est la probabilité que le programme soit réellement bénéfique ? Plutôt que de simplement dire “p <
0.001”, on peut estimer la distribution a posteriori de la différence de taux de rétention entre groupes,
via une approche bayésienne.

Supposons une loi Beta–Binomiale : on assigne a priori à chaque taux de rétention (témoin et
programme) une loi Beta(1,1) non-informative (uniforme sur [0,1]). Après observation des données, la
loi a posteriori du taux de rétention témoin sera $Beta(\alpha_{T}, \beta_{T})$ avec $\alpha_{T} = 1 +
\text{(nb retenus témoins)}$ et $\beta_{T} = 1 + \text{(nb non-retenus témoins)}$. De même pour le
groupe programme ($\alpha_{P}, \beta_{P}$). On peut alors estimer la distribution a posteriori de la
différence $d = p_{P} - p_{T}$ par simulation de Monte Carlo.

In [6]:
import numpy.random as npr
# Paramètres postérieurs Beta
alpha_T = 1 + conv_témoin
beta_T = 1 + (n_témoin - conv_témoin)
alpha_P = 1 + conv_programme
beta_P = 1 + (n_programme - conv_programme)
# Tirages Monte Carlo
m = 100000
p_temoins = npr.beta(alpha_T, beta_T, size=m)
p_program = npr.beta(alpha_P, beta_P, size=m)
diff = p_program - p_temoins
# Estimation de la probabilité que le programme soit meilleur et intervalle de crédibilité
prob = (diff > 0).mean()
inf, sup = np.percentile(diff, [2.5, 97.5])
print("P(diff > 0) = {:.3f}".format(prob))
print("IC crédible 95%: [{:.3f}, {:.3f}]".format(inf, sup))

P(diff > 0) = 1.000
IC crédible 95%: [0.051, 0.160]


On peut interpréter : « La probabilité a posteriori que le programme de fidélité augmente réellement la
rétention est ~99.9% (quasi-certitude). De plus, on estime avec 95% de crédibilité que l’augmentation réelle du
taux de rétention se situe entre +5 points et +16 points. » Cette formulation, issue de l’approche
bayésienne, est souvent plus parlante pour un marketeur qu’une p-value . On voit que l’intervalle
crédible exclut 0 et est plutôt large, reflétant l’incertitude résiduelle sur la magnitude exacte de l’effet.

## Discussion : hypothèses, limites, interprétation

Le cas de la fidélisation illustre une expérimentation aléatoire réussie. Les hypothèses pour valider les
conclusions sont :  
(1) la randomisation a bien été effectuée (pas de biais de sélection),  
(2) il n’y a pas eu
de contamination entre groupes (par ex, un client témoin n’a pu bénéficier indirectement du
programme),  
(3) la période de mesure est suffisante pour capturer l’effet, et  
(4) les deux groupes ont été
suivis dans des conditions identiques en dehors du traitement.  

**Sous ces conditions, on peut interpréter
la différence observée comme causale** : le programme de fidélité cause une augmentation du taux de
rétention d’environ 10 points dans cet échantillon.

**Limites :**  
Un test A/B isolé ne dit rien sur la pérennité de l’effet (quid après 12 mois ?) ni sur la rentabilité
(le programme a un coût).  
Il indique un effet moyen sur la population testée ; il est possible que l’effet
varie selon le profil de clients (il serait intéressant de segmenter par ancienneté, type de client…).  
Par ailleurs, la taille d’échantillon doit être suffisante pour détecter l’effet visé (on suppose que le plan
d’expérience a pris cela en compte a priori).  
Enfin, on a supposé ici un taux d’acceptation de 100% du
programme par les clients du groupe test.  
En pratique, si l’adhésion au programme était optionnelle,
l’effet mesuré serait un mélange d’un effet d’adoption et d’un effet du programme lui-même – cela
nécessite une analyse d’intention de traiter ou des méthodes d’ajustement (voir as sur le matching,
qui pourrait aussi s’appliquer ici pour comparer adhérents vs non-adhérents a posteriori si la
randomisation n’avait pas été possible).

En résumé, ce cas montre la puissance d’un **test contrôlé randomisé** : **c’est la méthode la plus solide
pour établir une causalité en marketing** .  
Toutefois, on souligne qu’il n’est pas toujours possible de
réaliser un tel test (problèmes éthiques, logistiques ou coûts) . Dans ces situations, on se tourne vers
des méthodes quasi-expérimentales.